In [8]:
!python -m pip install --upgrade pip

In [9]:
!pip install -q -U transformers onnx onnxruntime datasets sentencepiece kagglehub kaggle

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model = AutoModelForSequenceClassification.from_pretrained('norsu/ai-detector-test')
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implem

In [2]:
import torch
import time
model.eval()
start = time.time()
input = tokenizer("asdsf", return_tensors='pt')
with torch.inference_mode():
  print(torch.softmax(model(**input)[0],1))

total = time.time() - start
total

tensor([[8.8574e-04, 9.9911e-01]])


1.2911760807037354

In [3]:
torch.save(model, f="model.pt")

In [4]:
import os
pt_size = os.path.getsize("model.pt")/(1024**2)
pt_size

703.6260967254639

In [5]:
from datasets import load_dataset
import kagglehub
!kaggle datasets download -d shanegerami/ai-vs-human-text
# kagglehub.login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset URL: https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text
License(s): other
ai-vs-human-text.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import pandas as pd

data = pd.read_csv(r"AI_Human.csv")

In [6]:
data

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0
...,...,...
487230,Tie Face on Mars is really just a big misunder...,0.0
487231,The whole purpose of democracy is to create a ...,0.0
487232,I firmly believe that governments worldwide sh...,1.0
487233,I DFN't agree with this decision because a LFT...,0.0


In [7]:
data = data.astype({"generated":int})

In [8]:
from datasets import Dataset

data = Dataset.from_pandas(data[:1000])

In [9]:
data = data.rename_column("generated","label")

In [10]:
st = time.time()

inputs = tokenizer(data[:100]['text'], return_tensors="pt", padding=True, truncation=True, max_length = 512)
corr = 0
with torch.inference_mode():
    outputs = torch.argmax(model(**inputs)[0],1)
    for i in range(len(outputs)):
        if outputs[i] == data[i]['label']:
          corr +=1
    pt_acc = corr/len(outputs)
    print(pt_acc)
pt_time = time.time() - st
print(pt_time)

0.6
105.0856523513794


In [11]:
import onnx

batch_size = 32


with torch.inference_mode():
    inputs = {"input_ids":torch.ones(1, 512, dtype=torch.int64),
     "attention_mask":torch.ones(1, 512, dtype=torch.int64),}
    outputs = model(**inputs)
    symbolic_names = {0:'batch_size', 1:'max_seq_len'}
    torch.onnx.export(model,
                (inputs['input_ids'],
                  inputs['attention_mask'],),
                      r"lean-ai-detector.onnx",
                      opset_version=12,
                      input_names=['input_ids',
                                   'input_mask',],
                      output_names=['output'],
                      dynamic_axes={'input_ids': symbolic_names,
                                    'input_mask': symbolic_names,})



/usr/local/lib/python3.10/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:547: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(mid - 1).type_as(relative_pos),
/usr/local/lib/python3.10/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:551: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.ceil(torch.log(abs_pos / mid) / torch.log(torch.tensor((max_position - 1) / mid)) * (mid - 1)) + mid
/usr/local/lib/python3.10/dist-packages/

In [12]:
onnx_size = os.path.getsize("lean-ai-detector.onnx")/(1024**2)
onnx_size

704.3487310409546

In [13]:
import onnxruntime

model_path = "lean-ai-detector.onnx"

sess_options = onnxruntime.SessionOptions()
session = onnxruntime.InferenceSession(model_path, sess_options)

In [17]:
import numpy as np

inputs = tokenizer("mad", return_tensors="np")

ort_inputs = {
                    'input_ids':  inputs['input_ids'],
                    'input_mask': inputs['attention_mask'],
              }

logits = np.reshape(session.run(None, ort_inputs), (-1,2))
logits

array([[-3.838808,  3.19955 ]], dtype=float32)

In [18]:
st = time.time()

inputs = tokenizer(data[:100]['text'], return_tensors="np", padding=True, truncation=True,max_length=512)
ort_inputs = {
                    'input_ids':  inputs['input_ids'],
                    'input_mask': inputs['attention_mask'],
              }
outputs = np.argmax(np.reshape(session.run(None, ort_inputs), (-1,2)),1)
corr = 0
for i in range(len(outputs)):
    if outputs[i] == data[i]['label']:
        corr +=1
onnx_acc = corr/len(outputs)
print(onnx_acc)

onnx_time = time.time()-st
print(onnx_time)

0.6
153.32597064971924


In [19]:
onnx_model_path = "lean-ai-detector.onnx"
quantized_model_path = "Q-lean-ai-detector.onnx"
from onnxruntime.quantization import quantize_dynamic, QuantType
onnx_opt_model = onnx.load(onnx_model_path)
quantize_dynamic(onnx_model_path,
                quantized_model_path,
                weight_type=QuantType.QInt8)

In [21]:
quant_size = os.path.getsize("Q-lean-ai-detector.onnx")/(1024**2)
quant_size

232.57864952087402

In [26]:
model_path = "Q-lean-ai-detector.onnx"

sess_options = onnxruntime.SessionOptions()
session = onnxruntime.InferenceSession(model_path, sess_options)

st = time.time()

inputs = tokenizer(data[:100]['text'], return_tensors="np", padding=True, truncation=True, max_length=512)
ort_inputs = {
                    'input_ids':  inputs['input_ids'],
                    'input_mask': inputs['attention_mask'],
              }
outputs = np.argmax(np.reshape(session.run(None, ort_inputs), (-1,2)),1)
corr = 0
for i in range(len(outputs)):
    if outputs[i] == data[i]['label']:
        corr +=1
quant_acc = corr/len(outputs)
print(quant_acc)

quant_time = time.time() - st
print(quant_time)

0.0
146.7533175945282


In [41]:
a = tokenizer("asdf", return_tensors="np")

ort_inputs = {
                    'input_ids':  a['input_ids'],
                    'input_mask': a['attention_mask'],
                }

In [42]:
import numpy as np

logits = np.reshape(session.run(None, ort_inputs), (-1,2))

In [46]:
np.argmax(logits)

1